<a href="https://colab.research.google.com/github/JinHuiXu1991/Jin_DATA606/blob/main/ipynb/DATA606_Part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Amazon Product Recommender Systems
## Author: Jin Hui Xu

In [1]:
!wget https://github.com/JinHuiXu1991/Jin_DATA606/blob/main/cleaned_data/cleaned_amazon_product.zip?raw=true

!wget https://github.com/JinHuiXu1991/Jin_DATA606/blob/main/cleaned_data/cleaned_amazon_review.zip?raw=true

--2022-03-20 00:26:05--  https://github.com/JinHuiXu1991/Jin_DATA606/blob/main/cleaned_data/cleaned_amazon_product.zip?raw=true
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/JinHuiXu1991/Jin_DATA606/raw/main/cleaned_data/cleaned_amazon_product.zip [following]
--2022-03-20 00:26:05--  https://github.com/JinHuiXu1991/Jin_DATA606/raw/main/cleaned_data/cleaned_amazon_product.zip
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/JinHuiXu1991/Jin_DATA606/main/cleaned_data/cleaned_amazon_product.zip [following]
--2022-03-20 00:26:05--  https://raw.githubusercontent.com/JinHuiXu1991/Jin_DATA606/main/cleaned_data/cleaned_amazon_product.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.

In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import linear_kernel
import time
import numpy as np

##Content Based Recommender

### Description Based Recommender

In [3]:
product_df = pd.read_csv('/content/cleaned_amazon_product.zip?raw=true', compression='zip')

In [4]:
product_df.head()

,category,description,title,brand,feature,main_cat,date,price,asin,imageURLHighRes,dateYear,dateMonth,all_text,all_text_token,word_count
0,"Appliances Refrigerators, Freezers & Ice Makers",NaN,Tupperware Freezer Square Round Container Set ...,Tupperware,Each 3-pc. set includes two 7/8-cup/200 mL and...,Appliances,2008-11-19,NaN,7301113188,[],2008.0,11.0,"Appliances Refrigerators, Freezers & Ice Maker...","['appliances', 'refrigerators', 'freezers', 'i...",51
1,"Appliances Refrigerators, Freezers & Ice Makers",2 X Tupperware Pure & Fresh Unique Covered Coo...,2 X Tupperware Pure &amp; Fresh Unique Covered...,Tupperware,2 X Tupperware Pure & Fresh Unique Covered Coo...,Appliances,2016-06-05,3.62,7861850250,['https://images-na.ssl-images-amazon.com/imag...,2016.0,6.0,"Appliances Refrigerators, Freezers & Ice Maker...","['appliances', 'refrigerators', 'freezers', 'i...",117
2,Appliances Parts & Accessories,NaN,The Cigar - Moments of Pleasure,The Cigar Book,NaN,Amazon Home,NaN,150.26,8792559360,['https://images-na.ssl-images-amazon.com/imag...,NaN,NaN,Appliances Parts & Accessories The Cigar - Mo...,"['appliances', 'parts', 'accessories', 'cigar'...",6
3,Appliances Parts & Accessories,"Multi purpost descaler, especially suited to w...",Caraselle 2X 50G Appliance Descalene,Caraselle,NaN,Tools & Home Improvement,2014-12-17,NaN,9792954481,['https://images-na.ssl-images-amazon.com/imag...,2014.0,12.0,Appliances Parts & Accessories Multi purpost d...,"['appliances', 'parts', 'accessories', 'multi'...",29
4,Appliances Parts & Accessories Range Parts & A...,Full gauge and size beveled-edge; furnished wi...,EATON Wiring 39CH-SP-L Arrow Hart 1-Gang Chrom...,EATON Wiring,Returns will not be honored on this closeout i...,Tools & Home Improvement,2007-01-16,3.43,B00002N5EL,[],2007.0,1.0,Appliances Parts & Accessories Range Parts & A...,"['appliances', 'parts', 'accessories', 'range'...",69


In [5]:
product_df.shape

(30239, 15)

In [6]:
# Replace all NaN with an empty string
product_df = product_df.fillna('')
product_df.isnull().sum()

category           0
description        0
title              0
brand              0
feature            0
main_cat           0
date               0
price              0
asin               0
imageURLHighRes    0
dateYear           0
dateMonth          0
all_text           0
all_text_token     0
word_count         0
dtype: int64

In [7]:
# Define a TF-IDF Vectorizer object and remove all english stopwords
tfidf = TfidfVectorizer(stop_words='english')

In [8]:
# Construct the required TF-IDF matrix 
tfidf_matrix = tfidf.fit_transform(product_df['description'])

In [9]:
# Output the shape of tfidf_matrix
tfidf_matrix.shape

(30239, 78513)

Both linear_kernel and cosine_similarity methods produce the same result. We would like to apply both methods can see the performance of creating the cosine similarity matrix, and choose the faster one.

In [10]:
start = time.time()
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
print("Time taken: %s seconds" % (time.time() - start))

Time taken: 14.685054302215576 seconds


In [11]:
start = time.time()
cosine_sim2 = cosine_similarity(tfidf_matrix, tfidf_matrix)
print("Time taken: %s seconds" % (time.time() - start))

Time taken: 14.306867599487305 seconds


The performance results are very close, and the cosine_similarity result will be used

In [12]:
# Reverse mapping of indices and product id
indices = pd.Series(product_df.index, index=product_df['asin'].str.lower()).drop_duplicates()

In [13]:
# Function that takes in product id as input and gives recommendations 
def description_recommender(id, cosine_sim = cosine_sim2, df = product_df, indices = indices):
  # get the index of the product that matches the id
  idx = indices[id.lower()]

  # get the pairwsie similarity scores 
  # then convert it into a list of tuples as described above
  sim_scores = list(enumerate(cosine_sim[idx]))

  # sort the product based on the cosine similarity scores
  sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

  # get the scores of the 10 most similar product. Ignore the first one because it is the input product.
  sim_scores = sim_scores[1:11]

  # get the product indices
  product_indices = [i[0] for i in sim_scores]

  # return the top 10 most similar product
  return df['asin'].iloc[product_indices].tolist(), df['title'].iloc[product_indices].tolist()

In [14]:
product_df[product_df['title'].str.contains('refrigerator', case=False)].head(1)

,category,description,title,brand,feature,main_cat,date,price,asin,imageURLHighRes,dateYear,dateMonth,all_text,all_text_token,word_count
131,"Appliances Refrigerators, Freezers & Ice Makers",This compact mini cooler and warmer holds 17 l...,Coldmate MR-128 Mini Cooler/Warmer Deluxe Mini...,Coldmate,Press the cold button to cool to 40 F and hot ...,Appliances,2001-10-02,,B0001YH10C,['https://images-na.ssl-images-amazon.com/imag...,2001.0,10.0,"Appliances Refrigerators, Freezers & Ice Maker...","['appliances', 'refrigerators', 'freezers', 'i...",99


In [15]:
# get input id and title for the recommendation
input_id = product_df[product_df['title'].str.contains('refrigerator', case=False)].iloc[0]['asin']
input_title = product_df[product_df['title'].str.contains('refrigerator', case=False)].iloc[0]['title']

In [16]:
#Get recommendations for Coldmate MR-128 Mini Cooler/Warmer Deluxe Mini Refrigerator, input the product id 
asin, title = description_recommender(input_id)

In [17]:
print('Description Based Recommender Result for {}, {}: '.format(input_id, input_title))
for i in range(0, 10):
  print('{}. {}, {}'.format(i+1, asin[i], title[i]))

Description Based Recommender Result for B0001YH10C, Coldmate MR-128 Mini Cooler/Warmer Deluxe Mini Refrigerator: 
1. B00RNAH5OY, goFridge Mini Fridge Portable Electric Cooler
2. B004NEYPYQ, Frost-Free 4.3 Cu. Ft. Refrigerator/Freezer White
3. B00DQLKY42, Kegco CBC-303-SSC 121 Can Beverage Center
4. B001F7H4RY, PORTABLE COOLER WARMER MINI FRIDGE WINE BEER
5. B004VFGNQU, Haier HNSE045 Refrigerator/Freezer, 4.5 Cubic Feet
6. B00K08XX4C, KingsBottle Vibration Free Beverage &amp; Wine Refrigerator with Glassdoor - 143.3 pounds Combo Fridge with 100 Beverage Cans or Bottles + 28 Wine Bottles - Drinks &amp; Wine Dual Cooler, Temperature Control
7. B00XYM30SU, Urban Islands Stainless Steel Refrigerator by Bull Outdoor Products
8. B00NHDARGE, Danby DBC047D1BSSPR Silhouette Built-In Beverage Center, 4.7 Cubic Feet, Black/Stainless Steel/Glass
9. B001L2DUAS, Stackable Wine Rack-72 Bottles Modular Hardwood Wine Racks, Very Easy to Put Together
10. B00FGWS33S, Danby DBC039A1BDB 20&quot; Freestandi

### Metadata based Recommender